A minimal example of gradient ascent using TF 2.0. Let's begin with an image of clouds, and modify it such that artifacts begin to appear.

In [ ]:
!pip install -q tensorflow-gpu==2.0.0-alpha0

In [ ]:
import numpy as np
import tensorflow as tf

from IPython.display import clear_output
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing import image

A few utilities for reading and preprocessing images.

In [ ]:
# Download an image and read it into a NumPy array, 
def download(url):
    name = url.split("/")[-1]
    image_path = tf.keras.utils.get_file(name, origin=url)
    img = image.load_img(image_path)
    return image.img_to_array(img)

# Scale pixels to between (-1.0 and 1.0)
def preprocess(img):
    return (img / 127.5) - 1
  
# Undo the preprocessing above
def deprocess(img):
    img = img.copy()
    img /= 2.
    img += 0.5
    img *= 255.
    return np.clip(img, 0, 255).astype('uint8')

# Display an image
def show(img):
    plt.figure(figsize=(12,12))
    plt.grid(False)
    plt.axis('off')
    plt.imshow(img)

# https://commons.wikimedia.org/wiki/File:Flickr_-_Nicholas_T_-_Big_Sky_(1).jpg
url = 'https://storage.googleapis.com/applied-dl/clouds.jpg'
img = preprocess(download(url))
show(deprocess(img))

Build a feature extraction model using the [Keras Functional API](https://www.tensorflow.org/alpha/guide/keras/functional). It's pretty sweet!

In [ ]:
inception_v3 = tf.keras.applications.InceptionV3(weights='imagenet',
                                                 include_top=False)

In [ ]:
# We'll maximize the activations of these layers
names = ['mixed2', 'mixed3', 'mixed4', 'mixed5']
layers = [inception_v3.get_layer(name).output for name in names]

# Create our feature extraction model
feat_extraction_model = tf.keras.Model(inputs=inception_v3.input, outputs=layers)

Here's our forward pass.

In [ ]:
def forward(img):
  
    # Create a batch
    img_batch = tf.expand_dims(img, axis=0)
  
    # Forward the image through Inception, extract activations
    # for the layers we selected above
    return feat_extraction_model(img_batch)

That's it for feature extraction. Now let's define our loss function. It'll be the mean activation of each of the layers we chose above.

In [ ]:
def calc_loss(layer_activations):
  
    total_loss = 0
  
    for act in layer_activations:
    
    # In gradient ascent, we'll want to maximize this value
    # so our image increasingly "excites" the layer
    loss = tf.math.reduce_mean(act)

    # Normalize by the number of units in the layer
    loss /= np.prod(act.shape)
    total_loss += loss

    return total_loss

We can now run gradient ascent (this is similar to training a classifier, with a twist). We'll treat our input image as a weight matrix, and find the gradients of the loss with respect to it. Those gradients will have the same shape as the image (since there's one for each pixel) - so we can add them directly to the image to modify it. Rinse and repeat.

In [ ]:
# Convert our image into a variable for training
img = tf.Variable(img)

# Run a few iterations of gradient ascent
steps = 400

for step in range(steps):
  
    with tf.GradientTape() as tape:    
        activations = forward(img)
        loss = calc_loss(activations)
    
    # How cool is this? It's the gradient of the 
    # loss (how excited the layer is) with respect to the
    # pixels of our random image!
    gradients = tape.gradient(loss, img)

    # Normalize the gradients
    gradients /= gradients.numpy().std() + 1e-8 
  
    # Update our image by directly adding the gradients
    # (because they're the same shape!)
    img.assign_add(gradients)
  
    if step % 50 == 0:
        clear_output()
        print ("Step %d, loss %f" % (step, loss))
        show(deprocess(img.numpy()))
        plt.show()

# Let's see the result
# Notice we're calling .numpy() here, which 
# takes us from TensorFlow land -> NumPy land
clear_output()
show(deprocess(img.numpy()))

That's it! For the bells and whistles (there are many) you can explore this full [implementation](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/deepdream) in TensorFlow v1 (perfect for concepts, but you'll probably want to adapt the code to something more modern), and this helpful [notebook](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.2-deep-dream.ipynb) from Deep Learning with Python using the Keras reference implementation.